In [102]:
!pip install scikit-learn==1.5.2

In [103]:
import pandas as pd

data = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             20000 non-null  int64  
 1   date           20000 non-null  object 
 2   bedrooms       20000 non-null  int64  
 3   bathrooms      20000 non-null  float64
 4   sqft_living    20000 non-null  int64  
 5   sqft_lot       20000 non-null  int64  
 6   floors         20000 non-null  float64
 7   waterfront     20000 non-null  int64  
 8   view           20000 non-null  int64  
 9   condition      20000 non-null  int64  
 10  grade          20000 non-null  int64  
 11  sqft_above     20000 non-null  int64  
 12  sqft_basement  20000 non-null  int64  
 13  yr_built       20000 non-null  int64  
 14  yr_renovated   20000 non-null  int64  
 15  zipcode        20000 non-null  int64  
 16  lat            20000 non-null  float64
 17  long           20000 non-null  float64
 18  sqft_l

In [104]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

data = data.drop(columns=['id'])

data['year_sold'] = data['date'].str[:4].astype(int)
data = data.drop(columns=['date'])  # Drop original date column
selected_features = ['sqft_living', 'bedrooms', 'bathrooms', 'grade', 'condition', 'yr_built', 'zipcode']

X = data[selected_features] # Features
y = data['price'] # Target

# if 'zipcode' in X.columns:
#     X_new.loc[:, 'zipcode'] = LabelEncoder().fit_transform(X_new['zipcode'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

xgb_model.fit(X_train, y_train)

train_score = xgb_model.score(X_train, y_train)
test_score = xgb_model.score(X_test, y_test)

train_score, test_score


(0.9397580959783209, 0.7987246602592847)

In [106]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
new_data_path = "https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing_holdout_test_mini.csv"
new_data = pd.read_csv(new_data_path)

new_data['year_sold'] = new_data['date'].str[:4].astype(int)
new_data = new_data.drop(columns=['id', 'date'])

X_new = new_data[selected_features]

# if 'zipcode' in new_data.columns:
#     X_new.loc[:, 'zipcode'] = LabelEncoder().fit_transform(X_new['zipcode'])

predicted_prices = xgb_model.predict(X_new)

predicted_prices = predicted_prices.round(0)

results = pd.DataFrame({'price': predicted_prices})

results.to_csv("team8-module3-predictions.csv", index=False)

print("Predictions saved to predicted_prices.csv")


Predictions saved to predicted_prices.csv
